In [ ]:
%%capture
! pip install transformers datasets evaluate

In [ ]:
import sys
print(sys.version)

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [ ]:
%%capture
from datasets import load_dataset
imdb = load_dataset("imdb")

In [ ]:
print(imdb["train"][0])
print(imdb["test"][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
print(imdb["train"].shape, imdb["test"].shape)

(25000, 2) (25000, 2)


In [ ]:
imdb["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
import pandas as pd
X_train = pd.DataFrame(imdb["train"]['text'][:25000]).rename(columns={0: 'text'})
y_train = pd.DataFrame(imdb["train"]['label'][:25000]).rename(columns={0: 'target'})
X_test = pd.DataFrame(imdb["test"]['text'][:25000]).rename(columns={0: 'text'})
y_test = pd.DataFrame(imdb["test"]['label'][:25000]).rename(columns={0: 'target'})
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
del(imdb)

(25000, 1) (25000, 1) (25000, 1) (25000, 1)


In [ ]:
n=1000
n_per_label =  (y_train['target'].value_counts()/len(y_train)* n).round().astype(int)
indices_sample = []
for label, count in n_per_label.items():
    # Sélectionner les indices pour chaque label
    label_indices = y_train[y_train['target'] == label].sample(n=count, random_state=42).index
    indices_sample.extend(label_indices)

y_train_spl = y_train.iloc[indices_sample]
X_train_spl = X_train.iloc[indices_sample]

n=250
n_per_label =  (y_test['target'].value_counts()/len(y_test)* n).round().astype(int)
indices_sample = []
for label, count in n_per_label.items():
    # Sélectionner les indices pour chaque label
    label_indices = y_test[y_test['target'] == label].sample(n=count, random_state=42).index
    indices_sample.extend(label_indices)

y_test_spl = y_test.iloc[indices_sample]
X_test_spl = X_test.iloc[indices_sample]

print(y_train_spl.shape, X_train_spl.shape, y_test_spl.shape, X_test_spl.shape)

(1000, 1) (1000, 1) (250, 1) (250, 1)


In [ ]:
from datasets import Dataset
df_train = {'text' : X_train_spl.text.tolist(), 'label': y_train_spl.target.tolist()}
df_test = {'text' : X_test_spl.text.tolist(), 'label': y_test_spl.target.tolist()}

df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [ ]:
%%capture
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
%%capture
tokenized_train = ds_train.map(preprocess_function, batched=True)
tokenized_test = ds_test.map(preprocess_function, batched=True)

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
%%capture
import evaluate
accuracy = evaluate.load("accuracy")

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# def compute_metrics(p):
#     predictions, labels = p
#     predictions = torch.argmax(torch.tensor(predictions), axis=1)
#     accuracy = (predictions == labels).sum().item() / len(labels)
#     return {"accuracy": accuracy}

In [ ]:
import torch
print(torch.cuda.is_available())
!nvidia-smi

True
Tue Mar  4 21:59:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             27W /   70W |   10416MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# "distilbert-base-uncased"  "huawei-noah/TinyBERT_General_4L_312D"  "albert-base-v2" "facebook/bart-base" "t5-small"
model_name= "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters:", f"{total_params:,}".replace(",", " ") )

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total number of parameters: 66 955 010


In [ ]:
torch.cuda.empty_cache()
!nvidia-smi --gpu-reset -i 0
#!kill -9 $(pgrep -f 'python')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
%%time
from transformers import AdamW, get_scheduler, Adafactor # LAMB,
from torch.optim import AdamW

# Commence avec un batch size modéré (8 ou 16) et surveille la mémoire GPU avec nvidia-smi.
# Si l'utilisation mémoire GPU est faible, augmente la taille du batch pour tirer parti du GPU, mais reste dans les limites de la mémoire.
# Si l'entraînement devient trop lent, réduis la taille du batch et teste l'effet sur les performances.
# Si la mémoire est saturée, utilise gradient_accumulation_steps pour simuler un plus grand batch tout en utilisant moins de mémoire.
torch.cuda.empty_cache() # pour vider le GPU

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to='none',
    fp16=True,
    logging_dir='./logs',
    logging_steps=1,
    logging_first_step=True,
    save_steps=100
)

# Créer un optimiseur AdamW personnalisé
# optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
# optimizer = LAMB(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
optimizer = Adafactor(model.parameters(), lr=training_args.learning_rate, relative_step=False, weight_decay=training_args.weight_decay)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pas de scheduler personnalisé ici
)

trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
results = trainer.evaluate()
print(results)